In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [24]:
df = pd.read_csv("insurance_data_2.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [26]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [27]:
X_train_scaled[:5]

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


In [28]:
X_test_scaled[:5]

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1


### Sigmoid

In [29]:
def sigmoid(X):
   return 1/(1+np.exp(-X))

### Binary Cross-Entropy Loss (Log Loss)


In [30]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15  # to prevent log(0)
    y_predicted = np.clip(y_predicted, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_predicted) + (1 - y_true) * np.log(1 - y_predicted))


In [31]:
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    learning_rate = 0.5
    n = len(age)

    for i in range(epochs):
        # Step 1: Weighted sum
        weighted_sum = w1 * age + w2 * affordibility + bias
        
        # Step 2: Apply sigmoid to get predictions
        y_predicted = sigmoid(weighted_sum)

        # Step 3: Calculate loss
        loss = log_loss(y_true, y_predicted)

        # Step 4: Compute gradients
        w1_grad = (1/n) * np.dot(age.T, (y_predicted - y_true))
        w2_grad = (1/n) * np.dot(affordibility.T, (y_predicted - y_true))
        bias_grad = np.mean(y_predicted - y_true)

        # Step 5: Update parameters
        w1 -= learning_rate * w1_grad
        w2 -= learning_rate * w2_grad
        bias -= learning_rate * bias_grad

        # Logging
        if i % 50 == 0 or loss <= loss_threshold:
            print(f"Epoch {i}: w1={w1:.4f}, w2={w2:.4f}, bias={bias:.4f}, loss={loss:.4f}")
        
        if loss <= loss_threshold:
            break

    return w1, w2, bias


In [32]:
# Predict class (0 or 1)
def predict(age, affordibility, w1, w2, bias):
    age = age / 100  
    weighted_sum = w1 * age + w2 * affordibility + bias
    return 1 if sigmoid(weighted_sum) >= 0.5 else 0

# Batch predictions
def batch_predict(X, w1, w2, bias):
    return [predict(age, aff, w1, w2, bias) for age, aff in zip(X['age']*100, X['affordibility'])]

# Accuracy
def accuracy(y_true, y_pred):
    return np.mean(np.array(y_true) == np.array(y_pred))


In [35]:
# Train model
w1, w2, bias = gradient_descent(
    X_train_scaled['age'].values,
    X_train_scaled['affordibility'].values,
    y_train.values,
    epochs=1000,
    loss_threshold=0.01
)

y_test_pred = batch_predict(X_test_scaled, w1, w2, bias)
print("Accuracy on test set:", accuracy(y_test.values, y_test_pred))


Epoch 0: w1=0.9749, w2=0.9483, bias=-0.1134, loss=0.7113
Epoch 50: w1=1.5033, w2=1.1084, bias=-1.2319, loss=0.5676
Epoch 100: w1=2.2007, w2=1.2942, bias=-1.6607, loss=0.5391
Epoch 150: w1=2.8496, w2=1.3697, bias=-1.9861, loss=0.5176
Epoch 200: w1=3.4430, w2=1.4042, bias=-2.2571, loss=0.5005
Epoch 250: w1=3.9825, w2=1.4239, bias=-2.4944, loss=0.4865
Epoch 300: w1=4.4722, w2=1.4388, bias=-2.7074, loss=0.4751
Epoch 350: w1=4.9172, w2=1.4526, bias=-2.9012, loss=0.4656
Epoch 400: w1=5.3226, w2=1.4665, bias=-3.0788, loss=0.4577
Epoch 450: w1=5.6926, w2=1.4808, bias=-3.2422, loss=0.4512
Epoch 500: w1=6.0313, w2=1.4955, bias=-3.3931, loss=0.4456
Epoch 550: w1=6.3422, w2=1.5104, bias=-3.5328, loss=0.4410
Epoch 600: w1=6.6281, w2=1.5253, bias=-3.6623, loss=0.4370
Epoch 650: w1=6.8918, w2=1.5400, bias=-3.7827, loss=0.4336
Epoch 700: w1=7.1355, w2=1.5546, bias=-3.8947, loss=0.4307
Epoch 750: w1=7.3612, w2=1.5688, bias=-3.9991, loss=0.4283
Epoch 800: w1=7.5707, w2=1.5826, bias=-4.0965, loss=0.4261
